In [1]:
#allows for working with urls
import requests

#allows html parsing
from bs4 import BeautifulSoup

#python data analyses
import pandas as pd

#allows for delays in requests
import time

In [3]:
#Get the Url for the scraped page
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
# standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"




#gets previous season(testing purpose only because season is all blank)
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
    
previous_season = soup.select("a.prev")[0].get("href")
standings_url = f"https://fbref.com{previous_season}"




data = requests.get(standings_url)
soup = BeautifulSoup(data.text)

#selects table of all teams
standings_table = soup.select('table.stats_table')[0]

#selects all 'a' refernces from the table
links = standings_table.find_all('a')

#finds all hyperlinks from the 'a' elements
links = [l.get("href") for l in links]

#finds all team links
links = [l for l in links if "/squads/" in l]

#gets entire team link url
team_urls = [f"https://fbref.com{l}" for l in links]


#selects the first team data
team_url = team_urls[0]
data = requests.get(team_url)

#selects the table that includes this string
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

soup = BeautifulSoup(data.text)

#selects all 'a' refernces from the table
links = soup.find_all('a')

#finds all hyperlinks from the 'a' elements
links = [l.get("href") for l in links]

#finds all team links
links = [l for l in links if l and "all_comps/shooting" in l]

#gets shooting link for this team
data = requests.get(f"https://fbref.com{links[0]}")

#gets shootings stats table
shooting = pd.read_html(data.text, match = "Shooting")[0]
#drops the top index level of table
shooting.columns = shooting.columns.droplevel()


#merge shooting table with match data
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
print(team_data)

          Date   Time              Comp                Round  Day    Venue  \
0   2023-08-06  16:00  Community Shield  FA Community Shield  Sun  Neutral   
1   2023-08-11  20:00    Premier League          Matchweek 1  Fri     Away   
2   2023-08-16  22:00         Super Cup       UEFA Super Cup  Wed     Home   
3   2023-08-19  20:00    Premier League          Matchweek 2  Sat     Home   
4   2023-08-27  14:00    Premier League          Matchweek 3  Sun     Away   
5   2023-09-02  15:00    Premier League          Matchweek 4  Sat     Home   
6   2023-09-16  15:00    Premier League          Matchweek 5  Sat     Away   
7   2023-09-19  20:00      Champions Lg          Group stage  Tue     Home   
8   2023-09-23  15:00    Premier League          Matchweek 6  Sat     Home   
9   2023-09-27  20:00           EFL Cup          Third round  Wed     Away   
10  2023-09-30  15:00    Premier League          Matchweek 7  Sat     Away   
11  2023-10-04  21:00      Champions Lg          Group stage  We

In [4]:
years = list(range(2024,2019, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
# standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data = team_data[team_data["Comp"] == "Serie A"]

        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

In [5]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

match_df.to_csv("matchesPrem(2024-2019).csv")